In [12]:
import pandas as pd 
import numpy as np
import json
import geopandas as gpd

In [13]:
df_all = pd.read_csv(r'C:\Users\User\Downloads\mobility_traces_complete_data\Final records at each stage\After type 3 without recursion\df_all_without_type_3_without_recursion_meth.csv')


In [14]:
df_all['datetime'] = pd.to_datetime(df_all['datetime'])

In [2]:
# reading shapefile of uttarakhand districts
shape_file_path = r"C:\Users\User\Downloads\Mobility_traces_new_data_code\GeoJSON files\India_Districts_map_JK_LD.geojson"
poly_file = gpd.read_file(shape_file_path)
shape_uttarakhand = poly_file["geometry"]
# shape_uttarakhand
# poly_file

In [98]:
poly_file.head(3)

id dt_code                  district st_code    year  \
0  None     639  North and Middle Andaman      35  2011_c   
1  None     640             South Andaman      35  2011_c   
2  None     638                  Nicobars      35  2011_c   

                 st_nm                                              layer  \
0  Andaman and Nicobar  andamannicobarislands andamannicobarislands_di...   
1  Andaman and Nicobar  andamannicobarislands andamannicobarislands_di...   
2  Andaman and Nicobar  andamannicobarislands andamannicobarislands_di...   

                                                path  \
0  C:/Users/Amit Agarwal/Google Drive/iitr_amit.c...   
1  C:/Users/Amit Agarwal/Google Drive/iitr_amit.c...   
2  C:/Users/Amit Agarwal/Google Drive/iitr_amit.c...   

                                            geometry  
0  MULTIPOLYGON (((92.75575 12.17525, 92.75538 12...  
1  MULTIPOLYGON (((92.52055 10.89783, 92.55898 10...  
2  MULTIPOLYGON (((93.89962 7.08159, 93.89832 7.0...

In [3]:
import geopandas as gpd
import pandas as pd

def stamp_states_and_districts(data: pd.DataFrame, shapefile_path: str) -> pd.DataFrame:
    """
    Assigns a state to each record based on its geographical coordinates.

    Parameters:
    - data: pd.DataFrame containing the data with columns 'latitude' and 'longitude'
    - shapefile_path: str, path to the shapefile of India states

    Returns:
    - pd.DataFrame with an added column 'state' indicating the state for each record
    """
    # Load the India states shapefile
    india_states_gdf = gpd.read_file(shapefile_path)
    
    # Convert the dataframe to a GeoDataFrame
    data_gdf = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data.longitude, data.latitude))
    
    # Ensure the CRS matches
    data_gdf.set_crs(epsg=4326, inplace=True)
    india_states_gdf = india_states_gdf.to_crs(epsg=4326)
    
    # Perform spatial join to find the state for each record
    data_with_states = gpd.sjoin(data_gdf, india_states_gdf, how='left', predicate='within')
    
    # Select relevant columns
    stamped_data = data_with_states[['maid', 'datetime', 'latitude', 'longitude', 'displacement', 'distance',
       'Velocity', 'st_nm','district','Nearest airport',
       'Coordinates of airport', 'Distance to airport (km)']]
    # [['maid', 'datetime', 'latitude', 'longitude', 'displacement', 'distance',
    #    'Velocity', 'st_nm','district','Nearest airport',
    #    'Coordinates of airport', 'Distance to airport (km)']] #, 'Action chosen', 'keep']]
    stamped_data=stamped_data.rename(columns={'st_nm':'state'})
    return stamped_data

import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

def fill_missing_states(df, target_column='state', feature_columns=['latitude', 'longitude'], n_neighbors=1):
    """
    Fill missing values in the DataFrame using KNN based on specified feature columns.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    target_column (str): The name of the column with missing values to fill.
    feature_columns (list): List of feature column names to use for KNN.
    n_neighbors (int): Number of neighbors to use for KNN.

    Returns:
    pd.DataFrame: DataFrame with missing values filled.
    """
    # Convert 'datetime' column to datetime type if present
    if 'datetime' in df.columns:
        df['datetime'] = pd.to_datetime(df['datetime'])

    # Separate rows with and without NaN in target column
    df_complete = df.dropna(subset=[target_column])
    df_missing = df[df[target_column].isna()]

    # Prepare training data for KNN
    X_train = df_complete[feature_columns]
    y_train = df_complete[target_column]

    # Initialize KNN classifier
    knn = KNeighborsClassifier(n_neighbors=n_neighbors)

    # Fit KNN model
    knn.fit(X_train, y_train)

    # Prepare data for prediction (only rows with missing values)
    X_missing = df_missing[feature_columns]

    # Predict missing values
    predicted_values = knn.predict(X_missing)

    # Fill missing values in the original DataFrame
    df.loc[df_missing.index, target_column] = predicted_values

    return df




In [93]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# Function to stamp districts against lat_lon_coordinates
def add_districts(df, poly_file):
    geometry = [Point(xy) for xy in zip(df.longitude, df.latitude)]
    coordinates_gdf = gpd.GeoDataFrame(df, geometry=geometry)
    print(f' coordinates_gdf shape: {coordinates_gdf.shape}')
    
    # Perform the spatial join
    joined_gdf = gpd.sjoin(coordinates_gdf, poly_file, how='left', predicate='within')
    print(f' joined_gdf shape: {joined_gdf.shape}')
    
    # Select relevant columns and rename them
    joined_gdf = joined_gdf[['maid', 'datetime', 'latitude', 'longitude', 'displacement', 'distance',
       'Velocity', 'state', 'name', 'Nearest airport',
       'Coordinates of airport', 'Distance to airport (km)', 'Action chosen', 'keep']]
    joined_gdf.rename(columns={'name': 'district'}, inplace=True)
    
    # Replace NaN values in district_id and district columns
    # joined_gdf.fillna({'district_id': 13, 'district': 'Outside'}, inplace=True)
    # print(f' joined_gdf shape after filling NaN: {joined_gdf.shape}')
    # Group by datetime and ensure only one entry per latitude-longitude pair per datetime
   #  grouped_gdf = joined_gdf.groupby(['maid','datetime', 'latitude', 'longitude']).first().reset_index()
    # Drop duplicates based on maid, datetime, latitude, and longitude, keeping the first occurrence
    grouped_gdf = joined_gdf.drop_duplicates(subset=['maid', 'datetime', 'latitude', 'longitude'], keep='first')
    print(f'grouped_gdf shape after dropping duplicates: {grouped_gdf.shape}')
    
    # Return the final DataFrame
    return grouped_gdf


def fill_missing_districts(df, target_column='district', feature_columns=['latitude', 'longitude'], n_neighbors=1):
    """
    Fill missing values in the DataFrame using KNN based on specified feature columns.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    target_column (str): The name of the column with missing values to fill.
    feature_columns (list): List of feature column names to use for KNN.
    n_neighbors (int): Number of neighbors to use for KNN.

    Returns:
    pd.DataFrame: DataFrame with missing values filled.
    """
    # Convert 'datetime' column to datetime type if present
    if 'datetime' in df.columns:
        df['datetime'] = pd.to_datetime(df['datetime'])

    # Separate rows with and without NaN in target column
    df_complete = df.dropna(subset=[target_column])
    df_missing = df[df[target_column].isna()]

    # Prepare training data for KNN
    X_train = df_complete[feature_columns]
    y_train = df_complete[target_column]

    # Initialize KNN classifier
    knn = KNeighborsClassifier(n_neighbors=n_neighbors)

    # Fit KNN model
    knn.fit(X_train, y_train)

    # Prepare data for prediction (only rows with missing values)
    X_missing = df_missing[feature_columns]

    # Predict missing values
    predicted_values = knn.predict(X_missing)

    # Fill missing values in the original DataFrame
    df.loc[df_missing.index, target_column] = predicted_values

    return df


In [135]:
import pandas as pd

def remove_unnecessary_timestamps(df):
    """
    Filters the dataframe to keep only the first and last records where latitude and longitude remain the same
    across consecutive timestamps for each maid.

    Parameters:
    df (pd.DataFrame): Input dataframe containing 'maid', 'datetime', 'latitude', and 'longitude' columns.

    Returns:
    pd.DataFrame: Filtered dataframe with only the first and last records where coordinates remain the same for each maid.
    """
    # Round latitude and longitude to 4 decimal places
    df['latitude'] = df['latitude'].astype(float).round(4)
    df['longitude'] = df['longitude'].astype(float).round(4)
    # Sort by 'maid' and 'datetime' to ensure consecutive timestamps are in order within each maid
    df = df.sort_values(by=['maid', 'datetime','latitude','longitude']).reset_index(drop=True)

    # Identify and keep first and last records where lat and long remain the same within each maid
    df['prev_lat'] = df.groupby('maid')['latitude'].shift(1)
    df['prev_long'] = df.groupby('maid')['longitude'].shift(1)
    df['next_lat'] = df.groupby('maid')['latitude'].shift(-1)
    df['next_long'] = df.groupby('maid')['longitude'].shift(-1)

    # Filter to keep rows where latitude or longitude changes within each maid
    filtered_df = df[(df['latitude'] != df['prev_lat']) |
                     (df['longitude'] != df['prev_long']) |
                     (df['latitude'] != df['next_lat']) |
                     (df['longitude'] != df['next_long'])]
    filtered_df.drop(['prev_lat', 'prev_long', 'next_lat', 'next_long'], axis=1, inplace=True)
    # Drop records for maids with only one record
    # Count occurrences of each maid
    maid_counts = df['maid'].value_counts()
    maid_single_records = maid_counts[maid_counts == 1].index
    filtered_df = filtered_df[~filtered_df['maid'].isin(maid_single_records)]

    return filtered_df



In [17]:
import warnings
warnings.filterwarnings('ignore')

In [18]:
df_all['maid'].nunique()

329076

In [156]:
df_all  = remove_unnecessary_timestamps(df_all)

In [ ]:
df_all.columns

Index(['maid', 'datetime', 'latitude', 'longitude', 'ACCURACY', 'DEVICE_OS',
       'displacement', 'distance', 'Velocity', 'dist_previous_next',
       'time_gap_previous_next', 'next_valid_index', 'keep',
       'prev_valid_index', 'Nearest airport', 'Coordinates of airport',
       'Distance to airport (km)', 'Added'],
      dtype='object')

In [ ]:
df_all= stamp_states_and_districts(df_all, shape_file_path)
df_all.head(3)

maid            datetime  latitude  \
0  000010b7-6bd5-4296-9235-2a824209b773 2022-09-01 03:34:47   30.5092   
1  000010b7-6bd5-4296-9235-2a824209b773 2022-09-01 03:34:48   31.1034   
2  000010b7-6bd5-4296-9235-2a824209b773 2022-09-01 06:58:02   30.3357   

   longitude  displacement    distance       Velocity             state  \
0    77.5428      0.000000    0.000000       0.000000  Himachal Pradesh   
1    77.1659     75.240993   75.240993  270867.574116  Himachal Pradesh   
2    77.8540    107.765970  183.006963      31.815441       Uttarakhand   

   district Nearest airport Coordinates of airport  Distance to airport (km)  
0   Sirmaur             NaN                    NaN                       NaN  
1    Shimla             NaN                    NaN                       NaN  
2  Dehradun             NaN                    NaN                       NaN

In [ ]:
df_all[df_all['maid'].isin(set(df_all[df_all['state'].isna()]['maid'].unique().tolist()))].count()

maid                        121470
datetime                    121470
latitude                    121470
longitude                   121470
displacement                121470
distance                    121470
Velocity                    121470
state                       115476
district                    115476
Nearest airport               4117
Coordinates of airport        4117
Distance to airport (km)      4117
dtype: int64

In [ ]:
df_all[df_all['state'].isna()].to_csv(r'C:\Users\User\Downloads\mobility_traces_complete_data\Final records at each stage\After type 3 without recursion\df_all_with_NaN_states.csv', index=False)

In [ ]:
UK_visitors = df_all[df_all['state']=='Uttarakhand']['maid'].unique().tolist()
non_visitors = df_all[~df_all['maid'].isin(set(UK_visitors))]['maid'].unique().tolist()
print(len(UK_visitors))
print(len(non_uk_visitors))

316989
3059


In [176]:
# Group by 'maid' and 'datetime' (date only), then count records
count_per_day_per_id = df_all.groupby(['maid', df_all['datetime'].dt.date]).size().reset_index(name='count')

# Pivot the table to have dates as columns and IDs as rows
pivot_df = count_per_day_per_id.pivot(index='maid', columns='datetime', values='count').fillna(0)

# Select the date columns for calculation
date_columns = pivot_df.columns

# Add summary statistics for the date columns
pivot_df['Total'] = pivot_df[date_columns].sum(axis=1)
pivot_df['Max'] = pivot_df[date_columns].max(axis=1)
pivot_df['Min'] = pivot_df[date_columns].min(axis=1)
pivot_df['Mean'] = pivot_df[date_columns].mean(axis=1)
pivot_df['Std. Deviation'] = pivot_df[date_columns].std(axis=1)
pivot_df.head()


datetime                              2022-09-01  2022-09-02  2022-09-03  \
maid                                                                       
000010b7-6bd5-4296-9235-2a824209b773        10.0        25.0        12.0   
00004eb1-6900-419c-b4be-0ec37b789989         3.0         2.0         1.0   
00005df5-30a2-4d15-ba7f-35bcd02f7f9d         0.0         0.0         1.0   
00007182-8140-450f-b91d-b4522543e0f6         0.0         5.0         2.0   
000097a4-0ed7-47a8-b01f-b2c727cbc993         0.0         0.0         0.0   

datetime                              2022-09-04  2022-09-05  2022-09-06  \
maid                                                                       
000010b7-6bd5-4296-9235-2a824209b773        12.0        12.0         8.0   
00004eb1-6900-419c-b4be-0ec37b789989         2.0         1.0         1.0   
00005df5-30a2-4d15-ba7f-35bcd02f7f9d         0.0         2.0         0.0   
00007182-8140-450f-b91d-b4522543e0f6         8.0         3.0         1.0   
000097a4-0ed7-47a8-b01f-b2c727cbc993         1.0         0.0         0.0   

datetime                              2022-09-07  2022-09-08  2022-09-09  \
maid                                                                       
000010b7-6bd5-4296-9235-2a824209b773        16.0         7.0        11.0   
00004eb1-6900-419c-b4be-0ec37b789989         0.0         0.0         3.0   
00005df5-30a2-4d15-ba7f-35bcd02f7f9d         0.0         0.0         0.0   
00007182-8140-450f-b91d-b4522543e0f6         0.0         2.0         4.0   
000097a4-0ed7-47a8-b01f-b2c727cbc993         0.0         2.0         0.0   

datetime                              2022-09-10  ...  2022-09-26  2022-09-27  \
maid                                              ...                           
000010b7-6bd5-4296-9235-2a824209b773        10.0  ...         8.0        12.0   
00004eb1-6900-419c-b4be-0ec37b789989         1.0  ...         5.0         5.0   
00005df5-30a2-4d15-ba7f-35bcd02f7f9d         0.0  ...         0.0         0.0   
00007182-8140-450f-b91d-b4522543e0f6         5.0  ...         0.0         0.0   
000097a4-0ed7-47a8-b01f-b2c727cbc993         0.0  ...         0.0         0.0   

datetime                              2022-09-28  2022-09-29  2022-09-30  \
maid                                                                       
000010b7-6bd5-4296-9235-2a824209b773        10.0         5.0         1.0   
00004eb1-6900-419c-b4be-0ec37b789989         1.0         1.0         2.0   
00005df5-30a2-4d15-ba7f-35bcd02f7f9d         0.0         0.0         1.0   
00007182-8140-450f-b91d-b4522543e0f6         0.0         0.0         0.0   
000097a4-0ed7-47a8-b01f-b2c727cbc993         0.0         0.0         1.0   

datetime                              Total   Max  Min       Mean  \
maid                                                                
000010b7-6bd5-4296-9235-2a824209b773  386.0  29.0  1.0  12.866667   
00004eb1-6900-419c-b4be-0ec37b789989   89.0  11.0  0.0   2.966667   
00005df5-30a2-4d15-ba7f-35bcd02f7f9d   12.0   6.0  0.0   0.400000   
00007182-8140-450f-b91d-b4522543e0f6   63.0  10.0  0.0   2.100000   
000097a4-0ed7-47a8-b01f-b2c727cbc993    7.0   2.0  0.0   0.233333   

datetime                              Std. Deviation  
maid                                                  
000010b7-6bd5-4296-9235-2a824209b773        6.542874  
00004eb1-6900-419c-b4be-0ec37b789989        2.870580  
00005df5-30a2-4d15-ba7f-35bcd02f7f9d        1.191927  
00007182-8140-450f-b91d-b4522543e0f6        3.122223  
000097a4-0ed7-47a8-b01f-b2c727cbc993        0.504007  

[5 rows x 35 columns]

In [15]:
# pivot_df = pd.read_csv(r'C:\Users\User\Downloads\mobility_traces_complete_data\Final records at each stage\After type 3 without recursion without non UK visitors\step_1_count_per_id_per_day.csv')

In [177]:
# pivot_df.to_csv(r'C:\Users\User\Downloads\mobility_traces_complete_data\Final records at each stage\After type 3 without recursion without non UK visitors\step_1_count_per_id_per_day.csv')


In [16]:
pivot_df.columns

Index(['maid', '2022-09-01', '2022-09-02', '2022-09-03', '2022-09-04',
       '2022-09-05', '2022-09-06', '2022-09-07', '2022-09-08', '2022-09-09',
       '2022-09-10', '2022-09-11', '2022-09-12', '2022-09-13', '2022-09-14',
       '2022-09-15', '2022-09-16', '2022-09-17', '2022-09-18', '2022-09-19',
       '2022-09-20', '2022-09-21', '2022-09-22', '2022-09-23', '2022-09-24',
       '2022-09-25', '2022-09-26', '2022-09-27', '2022-09-28', '2022-09-29',
       '2022-09-30', 'Total', 'Max', 'Min', 'Mean', 'Std. Deviation'],
      dtype='object')

In [178]:
df_all['datetime'].dt.date.unique()

array([datetime.date(2022, 9, 1), datetime.date(2022, 9, 2),
       datetime.date(2022, 9, 3), datetime.date(2022, 9, 4),
       datetime.date(2022, 9, 5), datetime.date(2022, 9, 6),
       datetime.date(2022, 9, 7), datetime.date(2022, 9, 8),
       datetime.date(2022, 9, 9), datetime.date(2022, 9, 10),
       datetime.date(2022, 9, 11), datetime.date(2022, 9, 12),
       datetime.date(2022, 9, 13), datetime.date(2022, 9, 14),
       datetime.date(2022, 9, 15), datetime.date(2022, 9, 16),
       datetime.date(2022, 9, 17), datetime.date(2022, 9, 18),
       datetime.date(2022, 9, 19), datetime.date(2022, 9, 20),
       datetime.date(2022, 9, 21), datetime.date(2022, 9, 22),
       datetime.date(2022, 9, 23), datetime.date(2022, 9, 24),
       datetime.date(2022, 9, 25), datetime.date(2022, 9, 26),
       datetime.date(2022, 9, 27), datetime.date(2022, 9, 28),
       datetime.date(2022, 9, 29), datetime.date(2022, 9, 30)],
      dtype=object)

In [179]:
pivot_df.columns

Index([      2022-09-01,       2022-09-02,       2022-09-03,       2022-09-04,
             2022-09-05,       2022-09-06,       2022-09-07,       2022-09-08,
             2022-09-09,       2022-09-10,       2022-09-11,       2022-09-12,
             2022-09-13,       2022-09-14,       2022-09-15,       2022-09-16,
             2022-09-17,       2022-09-18,       2022-09-19,       2022-09-20,
             2022-09-21,       2022-09-22,       2022-09-23,       2022-09-24,
             2022-09-25,       2022-09-26,       2022-09-27,       2022-09-28,
             2022-09-29,       2022-09-30,          'Total',            'Max',
                  'Min',           'Mean', 'Std. Deviation'],
      dtype='object', name='datetime')

In [20]:

# Define the threshold
X = 4


# Identify date columns that are datetime objects
# date_columns = [
#     '2022-09-01', '2022-09-02', '2022-09-03', '2022-09-04', '2022-09-05', 
#     '2022-09-06', '2022-09-07', '2022-09-08', '2022-09-09', '2022-09-10', 
#     '2022-09-11', '2022-09-12', '2022-09-13', '2022-09-14', '2022-09-15', 
#     '2022-09-16', '2022-09-17', '2022-09-18', '2022-09-19', '2022-09-20', 
#     '2022-09-21', '2022-09-22', '2022-09-23', '2022-09-24', '2022-09-25', 
#     '2022-09-26', '2022-09-27', '2022-09-28', '2022-09-29', '2022-09-30'
# ]
date_columns = [col.strftime('%Y-%m-%d') if isinstance(col, pd.Timestamp) else col for col in pivot_df.columns]

# Ensure date columns are sorted in chronological order
# date_columns = sorted(date_columns, key=lambda x: pd.to_datetime(x))
# print(f' date columns: {date_columns}')
# Function to check for four or more continuous days with counts less than X
def check_valid_distribution(row):
    # print(f'row: {row}')
    consecutive_days = 0
    for date in date_columns:
        # print(f' row date : {row[date]} and X= {X}')
        if row[date] < X:
            consecutive_days += 1
            if consecutive_days >= 4:
                # print(f' return false')
                return False
        else:
            consecutive_days = 0
    # print(f'returning True')
    return True

# Apply the function to each row
pivot_df['Valid distribution'] = pivot_df.apply(check_valid_distribution, axis=1)

# Display the DataFrame
# print(df)
pivot_df_all_valid_distribution = pivot_df[pivot_df['Valid distribution']==True]
pivot_df_all_valid_distribution.shape


TypeError: '<' not supported between instances of 'str' and 'int'

In [19]:
pivot_df['2022-09-01'].dtype

dtype('float64')

In [17]:
def description_of_valid_records(min_points_per_day, threshold_days, pivot_df, df_all):
        # Identify date columns that are datetime objects
    # date_columns = [
    #     '2022-09-01', '2022-09-02', '2022-09-03', '2022-09-04', '2022-09-05', 
    #     '2022-09-06', '2022-09-07', '2022-09-08', '2022-09-09', '2022-09-10', 
    #     '2022-09-11', '2022-09-12', '2022-09-13', '2022-09-14', '2022-09-15', 
    #     '2022-09-16', '2022-09-17', '2022-09-18', '2022-09-19', '2022-09-20', 
    #     '2022-09-21', '2022-09-22', '2022-09-23', '2022-09-24', '2022-09-25', 
    #     '2022-09-26', '2022-09-27', '2022-09-28', '2022-09-29', '2022-09-30'
    # ]
    date_columns = [col.strftime('%Y-%m-%d') if isinstance(col, pd.Timestamp) else col for col in pivot_df.columns]

    # Ensure date columns are sorted in chronological order
    # date_columns = sorted(date_columns, key=lambda x: pd.to_datetime(x))
    # print(f' date columns: {date_columns}')
    # Function to check for four or more continuous days with counts less than X
    def check_valid_distribution(row):
        # print(f'row: {row}')
        consecutive_days = 0
        if row['Total']>=60:
            for date in date_columns:
                # print(f' row date : {row[date]} and X= {X}')
                if row[date] < min_points_per_day:
                    consecutive_days += 1
                    if consecutive_days >= threshold_days:
                        # print(f' return false')
                        return False
                else:
                    consecutive_days = 0
        # print(f'returning True')
            return True
        else:
            return False

    # Apply the function to each row
    pivot_df['Valid distribution'] = pivot_df.apply(check_valid_distribution, axis=1)

    # Display the DataFrame
    # print(df)
    valid_ids = pivot_df[pivot_df['Valid distribution']==True].index.tolist()
    print(f' number of valid IDs : {len(valid_ids)}')
    corresponding_records = len(df_all[df_all['maid'].isin(set(valid_ids))])

    print(f'corresponding_records : {corresponding_records}')
    return valid_ids




In [18]:
valid_ids = description_of_valid_records(3, 4, pivot_df, df_all)  # valid Ids with no less than 3 points for at most 4 days
valid_records = df_all[df_all['maid'].isin(set(valid_ids))]

TypeError: '<' not supported between instances of 'str' and 'int'

In [189]:
x = valid_records[(valid_records['Velocity']>=150) & (valid_records['displacement']>=150)]
x.count() # 224066
x['maid'].nunique() #42959
valid_records.to_csv(r'C:\Users\User\Downloads\mobility_traces_complete_data\Final records at each stage\After type 3 without recursion without non UK visitors\step_2_valid_records_with_ok_dist.csv',index= False)

In [4]:
valid_records = pd.read_csv(r'C:\Users\User\Downloads\mobility_traces_complete_data\Final records at each stage\After type 3 without recursion without non UK visitors\step_2_valid_records_with_ok_dist.csv')

In [5]:
valid_records['maid'].nunique()

48197

In [6]:
valid_records.head()

Unnamed: 0                                  maid             datetime  \
0           0  000010b7-6bd5-4296-9235-2a824209b773  2022-09-01 03:34:47   
1           1  000010b7-6bd5-4296-9235-2a824209b773  2022-09-01 03:34:48   
2           2  000010b7-6bd5-4296-9235-2a824209b773  2022-09-01 06:58:02   
3           6  000010b7-6bd5-4296-9235-2a824209b773  2022-09-01 08:04:43   
4           7  000010b7-6bd5-4296-9235-2a824209b773  2022-09-01 08:37:01   

   latitude  longitude  displacement    distance       Velocity  \
0   30.5092    77.5428      0.000000    0.000000       0.000000   
1   31.1034    77.1659     75.240993   75.240993  270867.574116   
2   30.3357    77.8540    107.765970  183.006963      31.815441   
3   30.3357    77.8540      0.000000  183.006963       0.000000   
4   30.3688    77.8351      4.103091  187.110054       7.621841   

              state  district Nearest airport Coordinates of airport  \
0  Himachal Pradesh   Sirmaur             NaN                    NaN   
1  Himachal Pradesh    Shimla             NaN                    NaN   
2       Uttarakhand  Dehradun             NaN                    NaN   
3       Uttarakhand  Dehradun             NaN                    NaN   
4       Uttarakhand  Dehradun             NaN                    NaN   

   Distance to airport (km)  
0                       NaN  
1                       NaN  
2                       NaN  
3                       NaN  
4                       NaN

In [95]:
# valid_records = valid_records[['maid', 'datetime', 'latitude', 'longitude', 'ACCURACY', 'DEVICE_OS',
#        'displacement', 'distance', 'Velocity', 'keep', 'Nearest airport', 'Distance to airport (km)']]

In [162]:
x= df_all.iloc[:50]

In [201]:
IDs_with_NaN_states = valid_records[valid_records['state'].isna()]['maid'].unique().tolist()

In [203]:
valid_records[valid_records['state'].isna()].count()

Unnamed: 0                  2793
maid                        2793
datetime                    2793
latitude                    2793
longitude                   2793
displacement                2793
distance                    2793
Velocity                    2793
state                          0
district                       0
Nearest airport              107
Coordinates of airport       107
Distance to airport (km)     107
dtype: int64

In [204]:
valid_records = valid_records[~valid_records['maid'].isin(set(IDs_with_NaN_states))]
valid_records['maid'].nunique()

47915

In [205]:
valid_records.to_csv(r'C:\Users\User\Downloads\mobility_traces_complete_data\Final records at each stage\After type 3 without recursion without non UK visitors\step_4_valid_IDs_with_states_districts.csv', index=False)

In [2]:
valid_records= pd.read_csv(r'C:\Users\User\Downloads\mobility_traces_complete_data\Final records at each stage\After type 3 without recursion without non UK visitors\step_4_valid_IDs_with_states_districts.csv')

In [191]:
# valid_records[valid_records['state'].isna()].to_csv(r'C:\Users\User\Downloads\mobility_traces_complete_data\Final records at each stage\After type 3 without recursion without non UK visitors\step_4_valid_IDs_with_states_districts.csv', index=False)

In [3]:
valid_records.columns

Index(['Unnamed: 0', 'maid', 'datetime', 'latitude', 'longitude',
       'displacement', 'distance', 'Velocity', 'state', 'district',
       'Nearest airport', 'Coordinates of airport',
       'Distance to airport (km)'],
      dtype='object')

In [4]:
valid_records['uk_dist']= valid_records[(valid_records['state']=='Uttarakhand')]['district']
valid_records['uk_dist'] = valid_records['uk_dist'].fillna('Outside')
valid_records.head()

Unnamed: 0                                  maid             datetime  \
0           0  000010b7-6bd5-4296-9235-2a824209b773  2022-09-01 03:34:47   
1           1  000010b7-6bd5-4296-9235-2a824209b773  2022-09-01 03:34:48   
2           2  000010b7-6bd5-4296-9235-2a824209b773  2022-09-01 06:58:02   
3           6  000010b7-6bd5-4296-9235-2a824209b773  2022-09-01 08:04:43   
4           7  000010b7-6bd5-4296-9235-2a824209b773  2022-09-01 08:37:01   

   latitude  longitude  displacement    distance       Velocity  \
0   30.5092    77.5428      0.000000    0.000000       0.000000   
1   31.1034    77.1659     75.240993   75.240993  270867.574116   
2   30.3357    77.8540    107.765970  183.006963      31.815441   
3   30.3357    77.8540      0.000000  183.006963       0.000000   
4   30.3688    77.8351      4.103091  187.110054       7.621841   

              state  district Nearest airport Coordinates of airport  \
0  Himachal Pradesh   Sirmaur             NaN                    NaN   
1  Himachal Pradesh    Shimla             NaN                    NaN   
2       Uttarakhand  Dehradun             NaN                    NaN   
3       Uttarakhand  Dehradun             NaN                    NaN   
4       Uttarakhand  Dehradun             NaN                    NaN   

   Distance to airport (km)   uk_dist  
0                       NaN   Outside  
1                       NaN   Outside  
2                       NaN  Dehradun  
3                       NaN  Dehradun  
4                       NaN  Dehradun

In [5]:

#code to list down people who never travelled
# Identify unique districts visited by each person
unique_districts_per_person = valid_records.groupby('maid')['district'].nunique()
# print(f'unique_districts_per_person: {unique_districts_per_person}')
# Filter out people who visited only one district
people_never_traveled = unique_districts_per_person[unique_districts_per_person == 1].index
# print(f'people_never_traveled: {people_never_traveled}')
# Filter the original dataframe to include only people who never traveled
never_traveled_df = valid_records[valid_records['maid'].isin(people_never_traveled)]
# print(f'never_traveled_df: {never_traveled_df}')
# temp_df= never_traveled_df[never_traveled_df['district']=='Outside']
# print(f'never_traveled_df for outside people: {temp_df}')
# Count the number of people for each district who never traveled outside
count_never_traveled_per_district = never_traveled_df.groupby('district')['maid'].nunique()
count_never_traveled_per_district
# print("Count of people for each district who never traveled outside:")
# print(f'count_never_traveled_per_district: {count_never_traveled_per_district}')
# count_never_traveled_per_district.to_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\OD matrix\Count_of_non_travellers.csv')
# print(f'Save the records at {'/content/drive/MyDrive/Sample_Traces_data/OD matrix/People_stayed_within_districts.csv'}')
# count_never_traveled_per_district.to_csv(r'/content/drive/MyDrive/Sample_Traces_data/OD matrix/People_stayed_within_districts.csv')

district
Almora                 9
Bageshwar             12
Chamoli               12
Champawat              9
Dehradun             168
Haridwar              91
Nainital              75
Pauri Garhwal         17
Pithoragarh           30
Rudraprayag           14
Tehri Garhwal         11
Udham Singh Nagar     74
Uttarkashi             7
Name: maid, dtype: int64

In [9]:

#code to list down people who never travelled
# Identify unique states visited by each person
unique_states_per_person = valid_records.groupby('maid')['state'].nunique()
# print(f'unique_states_per_person: {unique_states_per_person}')
# Filter out people who visited only one state
people_never_traveled = unique_states_per_person[unique_states_per_person == 1].index
# print(f'people_never_traveled: {people_never_traveled}')
# Filter the original dataframe to include only people who never traveled
never_traveled_df = valid_records[valid_records['maid'].isin(people_never_traveled)]
# print(f'never_traveled_df: {never_traveled_df}')
# temp_df= never_traveled_df[never_traveled_df['state']=='Outside']
# print(f'never_traveled_df for outside people: {temp_df}')
# Count the number of people for each state who never traveled outside
count_never_traveled_per_state = never_traveled_df.groupby('state')['maid'].nunique()
intrastate_travellers = never_traveled_df.groupby('state')['maid'].unique().tolist()
# count_never_traveled_per_state
# print("Count of people for each state who never traveled outside:")
# print(f'count_never_traveled_per_state: {count_never_traveled_per_state}')
# count_never_traveled_per_state.to_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\OD matrix\Count_of_non_travellers.csv')
# print(f'Save the records at {'/content/drive/MyDrive/Sample_Traces_data/OD matrix/People_stayed_within_states.csv'}')
# count_never_traveled_per_state.to_csv(r'/content/drive/MyDrive/Sample_Traces_data/OD matrix/People_stayed_within_states.csv')

In [10]:
len(intrastate_travellers)

1

In [234]:
# intrastate_travellers_data = valid_records[valid_records['maid'].isin(set(intrastate_travellers))]
# intrastate_travellers_data.count()
intrastate_travellers_data.to_csv(r'C:\Users\User\Downloads\mobility_traces_complete_data\Final records at each stage\After type 3 without recursion without non UK visitors\step_5_valid_IDs_intrastate_travellers.csv', index=False)

In [215]:
valid_records.head()

maid             datetime  latitude  \
0  000010b7-6bd5-4296-9235-2a824209b773  2022-09-01 03:34:47   30.5092   
1  000010b7-6bd5-4296-9235-2a824209b773  2022-09-01 03:34:48   31.1034   
2  000010b7-6bd5-4296-9235-2a824209b773  2022-09-01 06:58:02   30.3357   
3  000010b7-6bd5-4296-9235-2a824209b773  2022-09-01 08:04:43   30.3357   
4  000010b7-6bd5-4296-9235-2a824209b773  2022-09-01 08:37:01   30.3688   

   longitude  displacement    distance       Velocity             state  \
0    77.5428      0.000000    0.000000       0.000000  Himachal Pradesh   
1    77.1659     75.240993   75.240993  270867.574116  Himachal Pradesh   
2    77.8540    107.765970  183.006963      31.815441       Uttarakhand   
3    77.8540      0.000000  183.006963       0.000000       Uttarakhand   
4    77.8351      4.103091  187.110054       7.621841       Uttarakhand   

   district Nearest airport Coordinates of airport  Distance to airport (km)  \
0   Sirmaur             NaN                    NaN                       NaN   
1    Shimla             NaN                    NaN                       NaN   
2  Dehradun             NaN                    NaN                       NaN   
3  Dehradun             NaN                    NaN                       NaN   
4  Dehradun             NaN                    NaN                       NaN   

    uk_dist  
0   Outside  
1   Outside  
2  Dehradun  
3  Dehradun  
4  Dehradun

In [125]:
never_traveled_df[never_traveled_df['district']=='Agra'].to_csv(r'C:\Users\User\Downloads\mobility_traces_complete_data\Final records at each stage\After type 3 without recursion\step_5_issues in non-travellers or intracity travellers\Ids with agra district.csv', index=False)

In [127]:
df_all= stamp_states_and_districts(df_all, shape_file_path)
df_all.head(3)
df_all.to_csv(r'C:\Users\User\Downloads\mobility_traces_complete_data\Final records at each stage\After type 3 without recursion\df_all_without_type_3_with_states_districts.csv', index= False)

In [128]:
df_all[df_all['state']=='Uttarakhand']['maid'].nunique()

316989

In [129]:
ids = df_all[df_all['state']=='Uttarakhand']['maid'].unique().tolist()

In [130]:
df_all[df_all['maid'].isin(set(ids))].count()

maid                        20164425
datetime                    20164425
latitude                    20164425
longitude                   20164425
displacement                20164425
distance                    20164425
Velocity                    20164425
state                       20159067
district                    20159067
Nearest airport              1109584
Distance to airport (km)     1109584
dtype: int64

In [131]:
df_all_raw = pd.read_csv(r"C:\Users\User\Downloads\mobility_traces_complete_data\Final records at each stage\Filtered_records\df_all_without_dups_sorted.csv")


In [138]:
df_all_raw.rename(columns= {'DEVICE_ID':'maid', 'EVENT_DATE':'datetime', 'LATITUDE':'latitude', 'LONGITUDE':'longitude'}, inplace=True)

In [140]:
df_all_raw = remove_unnecessary_timestamps(df_all_raw)

In [141]:
print(df_all_raw['maid'].nunique())
df_all_raw.count()

329076


maid         23286369
datetime     23286369
latitude     23286369
longitude    23286369
ACCURACY     23286369
DEVICE_OS    23284871
dtype: int64

In [144]:
df_all_raw = stamp_states_and_districts(df_all_raw, shape_file_path)

In [151]:
del uk_visitors_raw
gc.collect()

290

In [148]:
# uk_visitors_raw = df_all_raw[df_all_raw['state']=='Uttarakhand']['maid'].unique().tolist() #326017
non_uk_visitors = df_all_raw[~df_all_raw['maid'].isin(set(uk_visitors_raw))]['maid'].unique().tolist() #3059
len(non_uk_visitors)


3059

In [149]:
raw_data_non_uk_visitors = df_all_raw[df_all_raw['maid'].isin(set(non_uk_visitors))]
raw_data_non_uk_visitors.to_csv(r'C:\Users\User\Downloads\mobility_traces_complete_data\Final records at each stage\After type 3 without recursion\step_5_issues in non-travellers or intracity travellers\raw_data_non_uk_vis_for_new_shape_file.csv')

In [11]:

#code to list down people who never travelled
# Identify unique districts visited by each person
unique_districts_per_person = valid_records.groupby('maid')['district'].nunique()
# print(f'unique_districts_per_person: {unique_districts_per_person}')
# Filter out people who visited only one district
people_never_traveled = unique_districts_per_person[unique_districts_per_person == 1].index
# print(f'people_never_traveled: {people_never_traveled}')
# Filter the original dataframe to include only people who never traveled
never_traveled_df = valid_records[valid_records['maid'].isin(people_never_traveled)]
# print(f'never_traveled_df: {never_traveled_df}')
# temp_df= never_traveled_df[never_traveled_df['district']=='Outside']
# print(f'never_traveled_df for outside people: {temp_df}')
# Count the number of people for each district who never traveled outside
count_never_traveled_per_district = never_traveled_df.groupby('district')['maid'].nunique()
count_never_traveled_per_district
# print("Count of people for each district who never traveled outside:")
# print(f'count_never_traveled_per_district: {count_never_traveled_per_district}')
# count_never_traveled_per_district.to_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\OD matrix\Count_of_non_travellers.csv')
# print(f'Save the records at {'/content/drive/MyDrive/Sample_Traces_data/OD matrix/People_stayed_within_districts.csv'}')
# count_never_traveled_per_district.to_csv(r'/content/drive/MyDrive/Sample_Traces_data/OD matrix/People_stayed_within_districts.csv')

district
Almora                 9
Bageshwar             12
Chamoli               12
Champawat              9
Dehradun             168
Haridwar              91
Nainital              75
Pauri Garhwal         17
Pithoragarh           30
Rudraprayag           14
Tehri Garhwal         11
Udham Singh Nagar     74
Uttarkashi             7
Name: maid, dtype: int64